# 리뷰완료 2024.6.9

# 1. 글자 단위 RNN 언어 모델(Char RNNLM)

In [1]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [2]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
sentences = []
for sentence in f: # 데이터를 한 줄씩 읽는다.
    sentence = sentence.strip() # strip()을 통해 \r, \n을 제거한다.
    sentence = sentence.lower() # 소문자화.
    sentence = sentence.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

In [3]:
sentences[:5]

["*** start of the project gutenberg ebook alice's adventures in",
 'wonderland ***',
 '[illustration]',
 'alices adventures in wonderland',
 'by lewis carroll']

In [4]:
total_data = ' '.join(sentences)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(total_data))

문자열의 길이 또는 총 글자의 개수: 140323


In [5]:
print(total_data[:200])

*** start of the project gutenberg ebook alice's adventures in wonderland *** [illustration] alices adventures in wonderland by lewis carroll the millennium fulcrum edition 3.0 contents chapter i.    


In [6]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 43


# 글자를 정수(벡터가 아닌) 인덱스로 부여해주는 함수 정의 elaborated by paul

In [7]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, "'": 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, '0': 9, '3': 10, ':': 11, ';': 12, '?': 13, '[': 14, ']': 15, '_': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'k': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'w': 39, 'x': 40, 'y': 41, 'z': 42}


In [8]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [9]:
seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(total_data) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2338


In [10]:
train_X = []
train_y = []

for i in range(n_samples):
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 픽한다.
    X_sample = total_data[i * seq_length: (i + 1) * seq_length]

    # 정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트
    y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [11]:
print('X 데이터의 첫번째 샘플 :',train_X[0])
print('y 데이터의 첫번째 샘플 :',train_y[0])
print('-'*50)
print('X 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_X[0]])
print('y 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_y[0]])

X 데이터의 첫번째 샘플 : [5, 5, 5, 0, 35, 36, 17, 34, 36, 0, 31, 22, 0, 36, 24, 21, 0, 32, 34, 31, 26, 21, 19, 36, 0, 23, 37, 36, 21, 30, 18, 21, 34, 23, 0, 21, 18, 31, 31, 27, 0, 17, 28, 25, 19, 21, 2, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0]
y 데이터의 첫번째 샘플 : [5, 5, 0, 35, 36, 17, 34, 36, 0, 31, 22, 0, 36, 24, 21, 0, 32, 34, 31, 26, 21, 19, 36, 0, 23, 37, 36, 21, 30, 18, 21, 34, 23, 0, 21, 18, 31, 31, 27, 0, 17, 28, 25, 19, 21, 2, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25]
--------------------------------------------------
X 데이터의 첫번째 샘플 디코딩 : ['*', '*', '*', ' ', 's', 't', 'a', 'r', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'a', 'l', 'i', 'c', 'e', "'", 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ']
y 데이터의 첫번째 샘플 디코딩 : ['*', '*', ' ', 's', 't', 'a', 'r', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' '

In [12]:
print(train_X[1])

[25, 30, 0, 39, 31, 30, 20, 21, 34, 28, 17, 30, 20, 0, 5, 5, 5, 0, 14, 25, 28, 28, 37, 35, 36, 34, 17, 36, 25, 31, 30, 15, 0, 17, 28, 25, 19, 21, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25, 30, 0, 39, 31, 30, 20, 21, 34]


In [13]:
print(train_y[1])

[30, 0, 39, 31, 30, 20, 21, 34, 28, 17, 30, 20, 0, 5, 5, 5, 0, 14, 25, 28, 28, 37, 35, 36, 34, 17, 36, 25, 31, 30, 15, 0, 17, 28, 25, 19, 21, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25, 30, 0, 39, 31, 30, 20, 21, 34, 28]


In [14]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [15]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2338, 60, 43)
train_y의 크기(shape) : (2338, 60, 43)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 256)         307200    
                                                                 
 lstm_1 (LSTM)               (None, None, 256)         525312    
                                                                 
 time_distributed (TimeDist  (None, None, 43)          11051     
 ributed)                                                        
                                                                 
Total params: 843563 (3.22 MB)
Trainable params: 843563 (3.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
74/74 - 27s - loss: 3.0260 - accuracy: 0.1888 - 27s/epoch - 362ms/step
Epoch 2/80
74/74 - 32s - loss: 2.6697 - accuracy: 0.2617 - 32s/epoch - 435ms/step
Epoch 3/80
74/74 - 33s - loss: 2.3229 - accuracy: 0.3424 - 33s/epoch - 443ms/step
Epoch 4/80
74/74 - 33s - loss: 2.1828 - accuracy: 0.3783 - 33s/epoch - 443ms/step
Epoch 5/80
74/74 - 33s - loss: 2.0706 - accuracy: 0.4055 - 33s/epoch - 452ms/step
Epoch 6/80
74/74 - 35s - loss: 1.9890 - accuracy: 0.4254 - 35s/epoch - 478ms/step
Epoch 7/80


In [ ]:
def sentence_generation(model, length):
    # 글자에 대한 랜덤 인덱스 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤 익덱스로부터 글자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
result = sentence_generation(model, 100)
print(result)

# 2. 글자 단위 RNN(Char RNN)으로 텍스트 생성하기

이번에는 다 대 일(many-to-one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트 생성을 해보겠습니다.



In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

In [ ]:
tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)

In [ ]:
# 중복을 제거한 글자 집합 생성
char_vocab = sorted(list(set(raw_text)))
print(char_vocab)

In [ ]:
vocab_size = len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

In [ ]:
length = 11
sequences = []
for i in range(length, len(raw_text)):
    seq = raw_text[i-length:i] # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

In [ ]:
sequences[:10]

In [ ]:
encoded_sequences = []
for sequence in sequences: # 전체 데이터에서 문장 샘플을 1개씩 꺼낸다.
    encoded_sequence = [char_to_index[char] for char in sequence] # 문장 샘플에서 각 글자에 대해서 정수 인코딩을 수행.
    encoded_sequences.append(encoded_sequence)

In [ ]:
encoded_sequences[:5]

In [ ]:
encoded_sequences = np.array(encoded_sequences)
X_data = encoded_sequences[:,:-1]

# 맨 마지막 위치의 글자를 분리
y_data = encoded_sequences[:,-1]

In [ ]:
print(X_data[:5])
print(y_data[:5])

In [ ]:
# 원-핫 인코딩
X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

In [ ]:
print(X_data_one_hot.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs=100, verbose=2)

In [ ]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):

    # 초기 시퀀스
    init_text = seed_text
    sentence = ''

    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text] # 현재 시퀀스에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 패딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))

        # 입력한 X(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        
        for char, index in char_to_index.items():
            if index == result:
                break

        # 현재 시퀀스 + 예측 글자를 현재 시퀀스로 변경
        seed_text = seed_text + char

        # 예측 글자를 문장에 저장
        sentence = sentence + char

    sentence = init_text + sentence
    return sentence

In [ ]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))